## Introduction

Let's say that you are God and you are trying to create your next verdant paradise after the resounding success of that planet known as Earth. Some other hyperintelligenet species of humanoids are wandering around in the void and asking you to make an exact copy of Earth to put them on as they have been observing Earth for millions of years and they are very content with having a same environment as their new home, which means that they want the same amount of everything. For simplicity reasons let's say there are only two kinds of organisms, animals and plants. Now you being God, you do not have a concept of time as you live in a higher dimension, and as such you have no notion of memory. You don't remember what the distributions of animals and plants are back on Earth, and you will have to disrupt the lives of earthlings in order to examine them. However, you have in front you a ledger of all the conditional probabilities of an animal given a plant, and vice versa, since you wanted to make Earth a highly interconnected world. Now you want to save time so you want to draw one plant and one animal each time from the big messy pool that you have and put them on this new Earth, how would you go about doing that?
<img src="https://upload.wikimedia.org/wikipedia/commons/9/97/The_Earth_seen_from_Apollo_17.jpg" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/9/97/The_Earth_seen_from_Apollo_17.jpg

This is the problem of sampling, which is important in any kind of data analysis, and this situation in particular is sampling when you only have conditional distributions instead of a joint distribution of all the random variables. One algorithm to do this kind of sampling is Gibbs sampling. This has often been used when sampling from the joint distribution is difficult, either you don't know or calculating marginal distributions from conditional ones take too long, and as a means of statiscal inference. However, in order to understand Gibbs sampling, you have to know what Markov Chain Monte Carlo is and how it approximates the underlying joint distribution of where the data comes from. I will also talk briefly about why understanding how to sample is important, and the accept-reject algorithm.

## Motivating Example

Sampling is done wrong constantly in real life. When city planners try to measure whether roads are congested or not, they often just pick a random time in the day, thinking that they are picking a random sample from the distribution that they care about, and then drive at that hour. They then repeat this, and usually the conclusion is that no the roads are not congested, because normally they are only congested for small periods in the morning and the evening due to working hours, which is to be expected, and it is unlikely that the random times picked will hit these. However, this is not an informative experiment because they are mixing up what is part of the sample space. The sample space is cars, not time. They want to see whether a car is experiencing congestion, not whether a time period is congested. What they really should do is pick a random car and see if that car is driving on a congested road. Even though the periods of high congestion are short, they also involve the most cars. Therefore, if they pick cars at random many times they will get the correct measurement that if most cars they see are congested that means it is a problem, and if not then it isn't since most cars still drive during uncongested times.
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a4/Moscow_traffic_congestion.JPG" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/a/a4/Moscow_traffic_congestion.JPG



## Markov Chain Monte Carlo

Markov chains are essentially sequences where the probability of each event depends only on the event the precedes it. Markov Chain Monte Carlo is sampling from a distribution by constructing a Markov Chain. I will illustrate this concept with another example. Let's say you want to know the probability of raining in any given day. You don't have this information since you are not God. However, based on pass experiences, you do believe that P(rainy|yesterday was rainy) = 0.6, P(sunny|yesterday was rainy) = 0.4, P(rainy|yesterday was sunny) = 0.2, P(sunny|yesterday was sunny) = 0.8. For the purposes of illustration, as an outside observer I know that P(rainy) = 1/3 and P(sunny) = 2/3, and you can check that the conditional probabilities are correct. In this simple example you can obviously calculate the marginal probabilities by using Bayes Rule, but imagine if the marginal distribution is multivariate with many random variables, it would be hard to calculate.

If it's sunny today, what's the weather like tomorrow? Based on the conditional probabilities, it will be rainy with probability 0.2 and sunny with 0.8. What about the day after tomorrow? Assuming that it's sunny then, the sequences can either be rainy tomorrow, and then sunny, or two days of sunny weather, and if we assume the Markov property that the probability of every state only depends on the previous state, the probability of sunny weather is $$P(rainy|sunny)P(sunny|rainy) + P(sunny|sunny)P(sunny|sunny) = 0.2 * 0.4 + 0.8 * 0.8 = 0.72$$, and probability of rainy weather is 1 - 0.72 = 0.28. The conditional probabilities can also be thought of as transition probabilities in a markov chain, ie. the probability of transitioning to a state given that you are currently in some state, and they can be expressed as a matrix where each row is the previous state and the order of columns and rows are sunny and then rainy. $$T = \begin{bmatrix} 0.8 & 0.2\\0.4 & 0.6\end{bmatrix}$$
<img src="files/markov chain.jpg" style="width: 400px;"> Then the probability of the weather 2 days later is $$T * T = \begin{bmatrix} 0.72 & 0.28\\0.56 & 0.44\end{bmatrix}$$

If you keep doing this and find the probabilities of being sunny or rainy after n days as n approaches infinity, you can find that the probabilities get closer and closer to the true probabilities P(sunny) and P(rainy), regardless of which state you start with. Looking at the matrix, it will be $$\begin{bmatrix} \frac{2}{3}& \frac{1}{3}\\\frac{2}{3} & \frac{1}{3}\end{bmatrix}$$. The reason behind this is very complicated and involves proving that the transition probabilities satisfies a few properties and that the Markov Chain converges to a stationary distriubtion, but intuitively this makes sense if you look at the transition graph. Regardless of where you start, since the transition probabilities follow the underlying distribution, you will end up in a state with probability that reflect the true probabilities.
<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/Rain_Falling_over_Desert_at_Sunset.jpg" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/c/c7/Rain_Falling_over_Desert_at_Sunset.jpg

The code for Markov Chain Monte Carlo is very simple. You just keep matrix multiplying the previous result (initially being the transition itself) by the transition matrix for a number of iterations that you specify.

In [12]:
import numpy as np
def mcmc(t, iters):
    for i in t:
        row_sum = 0
        for j in i:
            if j < 0 or j > 1:
                raise(Exception("Transition probabilities have to be between 0 and 1!"))
            row_sum += j
        if row_sum < 1:
            raise(Exception("Transition probabilities of each row must add up to 1!"))
    if iters < 0:
        raise(Exception("Number of iterations has to be nonnegative!"))
    t_res = t
    for i in range(iters):
        t_res = t_res @ t
    return t_res

print(mcmc(np.asarray([[0.8, 0.2],[0.4, 0.6]]), 1000))

[[ 0.66666667  0.33333333]
 [ 0.66666667  0.33333333]]


Moreover, Markov Chain Monte Carlo is not only useful for approximating the marginal distribution, but also with a similar algorithm, you can draw a sample that has a probability of being selected approaching the marginal distribution, the longer you run it. Sampling by Markov Chain Monte Carlo is called Gibbs sampling.

## Gibbs Sampling

Let's demonstrate Gibbs Sampling by completing the example given in the introduction. The steps are:
1. Pick a random animal uniformly
2. Pick a random plant from the conditional distribution given the animal you've picked in step 1, and discard the plant picked in step 3 if it exists.
3. Pick a random animal from the conditional distribution given the plant you've picked in step 2, and discard the animal picked in step 1.
4. Go to step 2, or go to step 1 to pick another sample with the current pair if you are satisfied.
<img src="https://upload.wikimedia.org/wikipedia/commons/d/d9/Collage_of_Nine_Dogs.jpg" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/d/d9/Collage_of_Nine_Dogs.jpg

As we've seen, picking a random animal is like starting from an initial state, and then picking a random plant from the conditional distribution of the animal is like transitioning to the next state based on the transition probabilities, and if we keep doing this we know that as the number of iterations approaches infinity, the probability of the sample that we've picked approaches its true probability.

Gibbs sampling also applies to multivariate distributions, which is what we have here. It is important to note the difference between this example and the rainy day example. If we were to pick a random weather from the rainy day example, the steps are:
1. Pick a random weather uniformly
2. Pick a random weather from the conditional distribution given the weather you've picked in step 1, and discard the weather picked in step 1.
3. Go to step 2, or go to step 1 to pick another sample with the current weather if you are satisfied.

The difference is that in the single-variate example the conditional probabilities are for example P(rainy|sunny), which is a random variable given itself in the previous sample, whereas P(dog|tree) on the other hand is from two random variables. You never pick from the conditional distribution of a random variable given itself for the multivariate case because then this sample would approximate the marginal distribution of this random variable instead of the joint distribution of all random variables. The code for this is:

In [13]:
def gibbs(t, iters):
    for i in t:
        row_sum = 0
        for j in i:
            if j < 0 or j > 1:
                raise(Exception("Transition probabilities have to be between 0 and 1!"))
            row_sum += j
        if row_sum < 1:
            raise(Exception("Transition probabilities of each row must add up to 1!"))
    if iters < 0:
        raise(Exception("Number of iterations has to be nonnegative!"))
    choices = [0, 1] #where 0 is sunny and 1 is rainy
    weathers = ["sunny", "rainy"]
    weather = np.random.choice(choices)
    for i in range(iters):
        weather = np.random.choice(a = choices, p = t[weather])
    return weathers[weather]

print(gibbs(np.asarray([[0.8, 0.2],[0.4, 0.6]]), 1000))

sunny


Basically it is just you first uniformly sample a weather and then iteratively replace the weather sampled by a new weather sampled from the conditional distribution of weather given the previously sampled weather.

It is easy to adapt Gibbs sampling to be used for inference, ie. to predict the values of random variables given observations of some other random variables. We just need to treat the observed random variables as fixed and follow the same algorithm, because you still sample from the same conditional distributions. For example, in the God example let's say there's a third category of organisms called fungi, where also you remember the conditional probabilities involving them, and a rival God. He wants you to fail your mission of recreating an exact Earth, as he also has this vision (that means he will also follow the underlying joint distribution) and wants you to fail so he can take over. He already sampled a triplet but he's covering the animal and plant. Now he reveals to you that the fungus is Maitake, and if you can guess correctly what the pair of animal and plant is, he will let you continue with the task, otherwise he will take over. What should you do?
1. Pick a random animal uniformly.
2. Pick a random plant from the conditional distribution of plants given animals and fungi, where their values equal the animal you picked and the fungus observed, and discard your previous plant if it exists.
3. Pick a random animal from the conditional distribution of animals given plants and fungi, where their values equal the plant you picked and the fungus observed, and discard your previous animal if it exists.
4. Go to step 2, or exit and choose the pair that occured the most if you are satisfied.
<img src="https://upload.wikimedia.org/wikipedia/commons/6/60/Eikhaas.JPG" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/6/60/Eikhaas.JPG

You can see that basically the fungus choice never changes, and you just follow the same algorithm as before of picking animals and plants. The pick that is the pair that appeared the most while sampling will be a reasonable inference of what the pair is (that forms the triplet coming from the joint distribution) that gets better with the number of iterations. The inference here is essentially the maximum a posteriori estimator since based on the posterior distribution, which is the conditional distribution given the fungus, this pair has the maximum posterior probability. The code is:

In [15]:
def gibbs_inference(t1, t2, iters):
    #Originally the matrices would have 3 dimensions, but since one dimension (fixed) is fixed,
    #the remaining choices can be packed into two 2-dimensional matrices
    for t in [t1, t2]:
        for i in t:
            row_sum = 0
            for j in i:
                if j < 0 or j > 1:
                    raise(Exception("Transition probabilities have to be between 0 and 1!"))
                row_sum += j
            if row_sum < 1:
                raise(Exception("Transition probabilities of each row must add up to 1!"))
        if iters < 0:
            raise(Exception("Number of iterations has to be nonnegative!"))
    choices = [0, 1]
    animals = ["dog", "cat"]
    plants = ["tree", "flower"]
    animal = np.random.choice(choices)
    animal_counts = [0, 0]
    animal_counts[animal] += 1
    plant_counts = [0, 0]
    for i in range(iters):
        plant = np.random.choice(a = choices, p = t2[animal])
        plant_counts[plant] += 1
        animal = np.random.choice(a = choices, p = t1[plant])
        animal_counts[animal] += 1
    final_animal = np.argmax(animals)
    final_plant = np.argmax(plants)
    return animals[final_animal], plants[final_plant], "maitake"

print(gibbs_inference(np.asarray([[0.8, 0.2],[0.4, 0.6]]), np.asarray([[0.7, 0.3],[0.5, 0.5]]), 1000))

('dog', 'tree', 'maitake')


This code is just fixing the fungus and only using the parts of the original matrices that correspond to being given Maitake, and then sampling similarly as before and in the end returning the pair of animal and plant that occured the most.

## Accept-Reject Algorithm

You might be wondering, how is the numpy random choice function (which is basically how to sample from a specified distribution) implemented? It is by the accept-reject algorithm. I will illustrate it by examples. Imagine that you have some dice, how can you sample a number from 27 to 39 each with equal probability?

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a5/6sided_dice.jpg" style="width: 300px;">
Source: https://upload.wikimedia.org/wikipedia/commons/a/a5/6sided_dice.jpg

In [4]:
import math
import random
#You have a black box that throws a dice
#Remember we can't use the numpy random choice method since this is how it's implemented
def update_dice(dice, ind):
    dice[ind] += 1
    if dice[ind] == 7:
        dice[ind] = 1
        update_dice(dice, ind + 1)

def make_dice_map(outputs, num_dice):
    #for each output, give some dice combination that maps to it
    num_map = {}
    dice = []
    for i in range(num_dice):
        dice.append(1)
    for i in outputs:
        tup = tuple(dice)
        num_map[tup] = i
        update_dice(dice, 0)
    return num_map

def throw_die():
    #We can call the random function since this uses time instead of a distribution
    return random.randint(1, 6)

def accept_reject(outputs = list(range(27, 70))):
    output_len = len(outputs)
    #three dice are sufficient to cover the whole space
    num_dice = int(math.ceil((math.log(output_len, 6))))
    num_map = make_dice_map(outputs, num_dice)
    throws = []
    for i in range(num_dice):
        throws.append(throw_die())
    while tuple(throws) not in num_map:
        #reject
        throws = []
        for i in range(num_dice):
            throws.append(throw_die())
    #accept if dice combination is in the map, and since each combination can be chosen
    #uniformly, each output can be chosen with equal probability
    return num_map[tuple(throws)]

print(accept_reject())

39


Now let's extend this to sampling from a continuous distribution, where your only tool is still a random number generator not implemented by sampling from distributions.

In [8]:
import scipy.integrate as integrate
def beta_func(alpha, beta):
    return integrate.quad(lambda t: math.pow(t, alpha - 1) * math.pow((1 - t), beta - 1), 0, 1)[0]

def beta_dist(x, alpha, beta):
    if x < 0 or x > 1:
        return 0
    else:
        return (1 / beta_func(alpha, beta)) * math.pow(x, alpha - 1) * math.pow((1 - x), beta - 1)
    
def uniform_dist(lower, upper):
    #the random number generator; it's not done by distribution but by time
    return random.uniform(lower, upper)

def beta_sample(alpha, beta):
    if alpha <= 0:
        raise(Exception("Alpha has to be positive!"))
    if beta <= 0:
        raise(Exception("Beta has to be positive!"))
    x = uniform_dist(0, 1)
    prob = beta_dist(x, alpha, beta)
    prob_samp = uniform_dist(0, 1)
    while prob_samp > prob:
        #reject
        x = uniform_dist(0, 1)
        prob = beta_dist(x, alpha, beta)
        prob_samp = uniform_dist(0, 1)
    #accept; there's a probability from the beta distribution chance that each sample would be accepted
    #since you draw a prob_samp from a uniform distribution (RNG) and the probabiliity that it's less than
    #the beta probability is just equal to the beta probability (by its cdf)
    return x

print(beta_sample(2, 2))

0.15463700599117547


## Sources and Extra Material

1. Markov Chain Explanation and Intersting Example: https://jeremykun.com/2015/04/06/markov-chain-monte-carlo-without-all-the-bullshit/
2. Markov Chain Monte Carlo Wiki: https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo
3. Gibbs Sampling Wiki: https://en.wikipedia.org/wiki/Gibbs_sampling
4. Gibbs D&D Example: https://stats.stackexchange.com/questions/10213/can-someone-explain-gibbs-sampling-in-very-simple-words
5. MCMC Weather Example: https://stats.stackexchange.com/questions/165/how-would-you-explain-markov-chain-monte-carlo-mcmc-to-a-layperson
6. Gibbs School Example (Third Response): https://www.quora.com/In-laymans-terms-how-does-Gibbs-Sampling-work
7. Markov Chain Convergence Proof: http://www.tcs.hut.fi/Studies/T-79.250/tekstit/lecnotes_02.pdf
8. Beta Distribution Wiki: https://en.wikipedia.org/wiki/Beta_distribution
9. Accept-Reject Algorithm Wiki: https://en.wikipedia.org/wiki/Rejection_sampling
10. Continuous Uniform Distribution Wiki: https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)